<a href="https://colab.research.google.com/github/Datamin-D/movie_predict/blob/master/Movie_Titles_Webcrawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
from bs4 import BeautifulSoup
import openpyxl
import xlwt,xlrd
from google.colab import files
import time
import re
import pandas as pd
import pickle




In [0]:
1. feature값 비교. + 연도별로 파일 저장할 수 있게하고 오류 뜬 연도는 별도 표시.  오전 -피클에서 수집한거 100 프린트하고 하나하나 대조.(겹친거 없나)
2. 수집한 데이터 중간 중간에 저장하는 법?
3. 전처리 - 어떤 영화를 '성공'으로 정의할지.


In [0]:
#installing selenium, prepare to connect 
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Fetched 88.7 kB in 2s (41.8 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


In [0]:
#Boxoffice mojo > extract movie titles by years(skip pickle has the movie titles info)

# 수집할 영화 연도 범위
released_years = []
released_months = []

years = [released_years.append(i) for i in range(2005,2019)] #연
months = [released_months.append(i) for i in range(1,13)] #월

#for문의 내포. [명령 for i in range(시작,끝,step)]
#print(released_years)

# raw data 수집,parsing, 영화 title저장
movie_list = []
for year in released_years:
  for month in released_months:
    #print(genre)
    raw = requests.get("https://www.boxofficemojo.com/calendar/"+str(year)+"-"+str(month)+"-01/")
      #print(raw)해서 response 200이면 정상.
      #주소니까 str로 해줘야. 

      #print(raw.text)
      #.text하면 --> URL소스코드 알아내기

    html = BeautifulSoup(raw.text,"html.parser")
      #변수 = BeautifulSoup(소스코드, 'html.parser')-->태그 기준으로 파싱해줌
      #print(html)

  # 1. 컨테이너 
    container = html.select("div.a-section.a-spacing-none.mojo-schedule-release-details")
  # 2. Title    
    for cont in container:
      movie_title = cont.select_one("a.a-link-normal h3").text.strip()
      #자식 ">" 자손 " "(띄어쓰기) ex) ul.type01 > li
      #.strip-->양쪽 공백 제거

      movie_list.append(movie_title+" "+"("+str(year)+")")
      #태그이름.class, 태그이름#id
print(len(movie_list))

#movie list 고유값만 추리기. set성질 이용.
my_set = set(movie_list)
movie_titles = list(my_set)
print(len(movie_titles))

with open ("movie_titles.txt","wb") as f:
  pickle.dump(movie_titles,f)
#movie_titles 표현형식: 영화이름, (연도) 
movie_titles[0]

12134
9834


'All Together (2012)'

In [0]:
#Boxoffice mojo에서 수집한 movie_titles
files.download("movie_titles.txt") #로컬드라이브로 다운로드
with open("movie_titles.txt","rb") as f:
  movie_titles = pickle.load(f)
print(movie_titles[0])


All Together (2012)


In [0]:
#영화 이름, 연도 분리..-->영화 이름, 연도와 매칭..굳이?  이미 IMDb 검색 기능에 구현되어 있을 것. --> abort.
"""
indv_title = []
indv_year =  []
for i in range(len(movie_titles)):
  p = re.compile(r'([\'\",/.a-zA-z0-9_ :!?\+\-\&\#\$\@\%\-\=\^\`\(\)]*) (\([0-9]+\))') #정규표현식으로 영화 이름 분리
  target = movie_list[i]
  s = p.search(target) #match는 처음부터 매칭해보고 없으면 진행 안함. search는 문자열 전체를 검색.(앞이 매칭 안되어도.)
  indv_title.append(s.group(1)) #i번째영화이름
  indv_year.append(s.group(2))  #i번째 영화의 연도(인덱스로 찾을 수 있음.)
  m = p.search(target)
print(indv_year[0])
 #이 코드 바탕으로imdb검색한 후 이름,연도 매칭하도록 코딩ㄱ *
 """

'\nindv_title = []\nindv_year =  []\nfor i in range(len(movie_titles)):\n  p = re.compile(r\'([\'",/.a-zA-z0-9_ :!?\\+\\-\\&\\#\\$\\@\\%\\-\\=\\^\\`\\(\\)]*) (\\([0-9]+\\))\') #정규표현식으로 영화 이름 분리\n  target = movie_list[i]\n  s = p.search(target) #match는 처음부터 매칭해보고 없으면 진행 안함. search는 문자열 전체를 검색.(앞이 매칭 안되어도.)\n  indv_title.append(s.group(1)) #i번째영화이름\n  indv_year.append(s.group(2))  #i번째 영화의 연도(인덱스로 찾을 수 있음.)\n  m = p.search(target)\nprint(indv_year[0])\n #이 코드 바탕으로imdb검색한 후 이름,연도 매칭하도록 코딩ㄱ *\n '

In [0]:
# Searching movie webpages in IMDb -->needs to be fixed. better idea? :title 밑에 있는 거로 클릭하도록. 

#Features 목록
Columns = ["Movie_title","Genre","Rate","Run_time","Budget","Gross_USA","Language","Directors","Writers","Stars","Certification"]

#DataFrame생성
df2 = pd.DataFrame(columns=Columns)

#영화 페이지 접근
for i in range(len(movie_titles[0:999])):#실제코드
#for i in range(6): #실험용
  url = "https://www.imdb.com/find?q="+str(movie_titles[i])+"&ref_=nv_sr_sm" #movie_title에 영화 이름 1개 들어가야함. 
  #print("real영화 제목:",movie_titles[i])
  wd.get(url)
  #print(wd.page_source)  # results
  try:
    click_movie = wd.find_elements_by_css_selector('td.result_text > a') #영화 검색창 후 이름 클릭
    click_movie[0].click() #클릭이 안되는 경우, no result found. pass하게 코딩.
  
    time.sleep(1)
  except IndexError: #영화 정보가 없어서 클릭이 안되면.
    continue 
  # 여기부터 영화 개별 웹페이지 진입 성공, 자료수집  

#Movie_title
  Movie_title = []
  try:
    imdb_title = wd.find_element_by_css_selector("#title-overview-widget > div.vital > div.title_block > div > div.titleBar > div.title_wrapper > h1").text
    if imdb_title == movie_titles[i]:
      Movie_title = imdb_title
    else:
      continue #이렇게 루프 뛰어넘어도 df index에는 공백으로 흔적 남아있음
    #print(wd.current_url)
  except NoSuchElementException:
    Movie_title.append(movie_titles[i])    
  #print(Movie_title)
    

#Genre - 수집 중 날짜까지 리스트 마지막에 수집되어서 index[-1]지우는 식으로 코딩함. 성공!
  Genre = []
  try:
    genre = wd.find_elements_by_css_selector("div.subtext a")
    for gen in range(len(genre)-1):
      Genre.append(genre[gen].text)
  except:
    Genre = "NA"

#Rate(전체 별점) 
  Rate = []
  try:
    Rate.append(wd.find_element_by_css_selector("div.ratingValue span:nth-of-type(1):first-child").text) # nth-of-type(n번째) n은 1부터 시작, first-child하면 부모 모든 요소 중 첫번째 자식만 선택 
  except NoSuchElementException:
    Rate.append("NA")

#Run_time
  Run_time = []
  try:
    Run_time.append(wd.find_element_by_css_selector('#title-overview-widget > div.vital > div.title_block > div > div.titleBar > div.title_wrapper > div > time').text)
  except NoSuchElementException:
    continue

   

#선택자#titleDetails >.txt-block
  Bcontainer = wd.find_elements_by_css_selector("#titleDetails >.txt-block")
#Budget **손보기
  Budget = "NA"
  try:
    for ind_Budget in range(len(Bcontainer)):
      if "Budget:" in Bcontainer[ind_Budget].text: #if문에 해당하는 내용이 없다해서 오류가 일어나지는 않음.
        budget = Bcontainer[ind_Budget].text
        Budget = budget.split(":")[1]
        break  #break는 조건이 참이면 반복문 종료. cf)continue는 아래의 코드를 실행않고 다음 반복으로 넘어감.
      else:
        Budget = "NA"
  except:
    Budget = "NA"
#Gross_USA
  Gross_USA = "NA"
  try:
    for ind_gusa in range(len(Bcontainer)):
      if "Gross USA:" in Bcontainer[ind_gusa].text:
        gusa = Bcontainer[ind_gusa].text
        Gross_USA.append(gusa.split(":")[1])
        #print(Gross_USA)
        break
      else:
        Gross_USA.append("NA")
  except:
    Gross_USA = "NA"
  
#Language: 로 시작하는 컨테이너찾기, 적당히 인덱싱해서 Language찾기. 
  Language = []
  try:
    find_language = wd.find_elements_by_css_selector("#titleDetails > div.txt-block")
    for ind_lang in range(len(find_language)):
      if find_language[ind_lang].text.strip().startswith("Language:"):
        Lang = find_language[ind_lang].text.strip()
    Language =  Lang[9:].strip().split("|")
    
  except NoSuchElementException:
    Language.append("NA")
  wd.close #개별 영화 페이지 닫기
  
#See more credits접근
  indv_movie_url = wd.current_url # 개별 영화 웹페이지 url
  target = str(indv_movie_url)
  p = re.compile(r"(https://www.imdb.com/title/)([a-zA-z]{2}[0-9]{7})(/\?ref_=fn_al_tt_1)")
  m = p.match(target)
  movie_code = m.group(2) #영화코드
  url = "https://www.imdb.com/title/"+movie_code+"/fullcredits/?ref_=tt_ov_st_sm"
  wd.get(url) #See full cast & crew 접근
  cu = wd.current_url

#Directors
  Directors_ch = []
  Directors = []
  try:
    director = wd.find_elements_by_css_selector("#fullcredits_content > table:nth-child(2) > tbody>tr")
    for ind_directors in range(len(director)):
      Directors_ch.append(director[ind_directors].text)
    for ind_Directors in range(len(Directors_ch)):
      target = Directors_ch[ind_Directors]
      p = re.compile("[\-. a-zA-z0-9]+")
      m = p.match(target)
      
      Director = m.group().strip()
      Directors.append(Director) #director_list
    #print(Directors) 
  except AttributeError:
    Directors.append("NA")

# Writers
  Writers = []
  try:
    Writing_credits = wd.find_elements_by_css_selector("#fullcredits_content > table:nth-child(4) > tbody > tr > td.name > a") #Writing Credits 박스 접근. 아래 자식으로 작가 이름 크롤링ㄱ 
    for ind_writer in range(len(Writing_credits)):
      Writers.append(Writing_credits[ind_writer].text)
  except:
    Writers = "NA"

  #if #fullcredits_content h4.dataHeaderWithBorder == Writing Credits 거기 멈춰서 안에 있는 박스 > ..는 복잡.
  #걍 nth-ofchild 기능으로 선택자 처리함.

  # Stars 상위16명까지. -->ㅡmin (2003)시도 완료
  #홀수 번호 배우들 수집 
  Stars_ch = []
  Stars_odd = []
  Stars_even = []
  Stars = []

  try:
    container_odd = wd.find_elements_by_css_selector("#fullcredits_content > table.cast_list > tbody > tr.odd") #홀수번호 배우들 데이터 수집
    for ind_oddstar in range(9): #8번째 홀수까지 반복, 15
      Stars_odd.append(container_odd[ind_oddstar].text)
  except IndexError: 
    pass

  #짝수 번호 배우들 수집
  try:
    container_even = wd.find_elements_by_css_selector("#fullcredits_content > table.cast_list > tbody > tr.even") #짝수번호 배우들 데이터 수집
    for ind_evenstar in range(9): #8번째 짝수까지 반복,16 
      Stars_even.append(container_even[ind_evenstar].text)
  except IndexError: 
    pass
  #홀,짝 합치기
  try:

    for ind_star in range(8):
      Stars_ch.append(Stars_odd[ind_star])
      Stars_ch.append(Stars_even[ind_star])
    #print(Stars_ch) 합치기 완료
  except IndexError: #배우 15명 미만일 시 pass.
    pass

  
  #배우"...캐릭터"지우기는 정규 표현식으로 처리.
  try:
    for ind_ch in range(len(Stars_ch)):
      target = Stars_ch[ind_ch]
      p = re.compile(r"([\-\.\' a-zA-z0-9]+)( ... )([\-\.\' a-zA-z0-9]+)")
      m = p.match(target)
      Star = m.group(1).strip()
      Stars.append(Star) #배우목록
      #print(Star)
  except AttributeError:
    Star
  wd.close #star닫기

#Certificate -->손보기
  Certificate = []
  #indv_movie_url는  개별 영화 웹페이지 url
  target = str(indv_movie_url)
  p = re.compile(r"(https://www.imdb.com/title/)([a-zA-z]{2}[0-9]{7})(/\?ref_=fn_al_tt_1)")
  m = p.match(target)
  movie_code = m.group(2) #영화코드
  #Certificate > clicking see more button using Selenium
  url = "https://www.imdb.com/title/"+movie_code+"/parentalguide?ref_=tt_stry_pg#certification" #개별영화 페이지 movie_title에 영화 이름 1개 들어가야함. 
  wd.get(url)
  #컨테이너 li.ipl-inline-list__item a
  countries = wd.find_elements_by_css_selector("tr#certifications-list > td > ul.ipl-inline-list >li.ipl-inline-list__item > a")
  try:
    for c_ind in range(len(countries)):
      if "United States" in countries[c_ind].text:
        US_cert = countries[c_ind].text
        nation_cert = US_cert.split(":")
        Certificate.append(nation_cert[1])
      else:
        pass #다른 나라들은 pass. 
  except:
    print("뭔가 에러남")
  #print(Certificate)
  wd.close #Certificate닫기
  #print(i,Movie_title[i],Genre,Rate,Run_time,Budget,Gross_USA,Language,Writers,Stars,Certificate)
  df2.loc[i] = Movie_title,Genre,Rate,Run_time,Budget,Gross_USA,Language,Directors,Writers,Stars,Certificate #i번째 영화, 그에 해당하는 데이터
  #print(Movie_title,Genre,Rate,Run_time,Budget,Gross_USA,Language,Directors,Writers,Stars,Certificate)
  
  time.sleep(1)


#print(df2)
df2.to_csv("movie_data_pilot.csv")

MaxRetryError: ignored

[1]


In [0]:
 #합칠 때 디버깅 실험실 (무시하셔도 됩니다!)

#Features 목록
Columns = ["Movie_title","Genre","Rate","Run_time","Budget","Gross_USA","Language","Directors","Writers","Stars","Certification"]

#DataFrame생성
df2 = pd.DataFrame(columns=Columns)

#영화 페이지 접근
#for i in range(len(movie_titles)):#실제코드
for i in range(3): #실험용
  url = "https://www.imdb.com/find?q="+str(movie_titles[i])+"&ref_=nv_sr_sm" #movie_title에 영화 이름 1개 들어가야함. 
  print("real영화 제목:",movie_titles[i])
  wd.get(url)
  #print(wd.page_source)  # results
  try:
    click_movie = wd.find_elements_by_css_selector('td.result_text > a') #영화 검색창 후 이름 클릭
    click_movie[0].click() #클릭이 안되는 경우, no result found. pass하게 코딩.
  
    time.sleep(1)
  except IndexError: #영화 정보가 없어서 클릭이 안되면.
    continue 
  # 여기부터 영화 개별 웹페이지 진입 성공, 자료수집  
  Movie_title = []
#Movie_title
  try:
    imdb_title = wd.find_element_by_css_selector("#title-overview-widget > div.vital > div.title_block > div > div.titleBar > div.title_wrapper > h1").text
    if imdb_title == movie_titles[i]:
      Movie_title = imdb_title
    else:
      continue #이렇게 루프 뛰어넘어도 df index에는 공백으로 흔적 남아있음
    #print(wd.current_url)
  except NoSuchElementException:
    Movie_title.append(movie_titles[i])    
  print(Movie_title)

  wd.close

real영화 제목: Jim Henson's Holiday Special with Fraggle Rock and Emmet Otter (2018)
real영화 제목: Racetime (2018)
Racetime (2018)
real영화 제목: Spider-Man: Into the Spider-Verse (2018)
Spider-Man: Into the Spider-Verse (2018)


In [0]:
#선택자 실험실.Requests, Selenium + Writers Stars 다 수집 다시. (무시하셔도 됩니다!)
"""
#requests
raw = requests.get("https://www.imdb.com/title/tt7286456/?ref_=ttrel_rel_tt") #test영화2"min" https://www.imdb.com/title/tt0360796/?ref_=fn_al_tt_2
html = BeautifulSoup(raw.text, "html.parser")# 파싱하는법 BeautifulSoup(request로 가져온 url.text, html.parser)
#퍼올꺼 = html.select_one("선택자").text.strip() 
"""
#selenium
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
url = "https://www.imdb.com/find?q="+str("joker (2019)")+"&ref_=nv_sr_sm" #영화 이름으로 검색
wd.get(url)
#wd.find_element_by_css_selector("tag")일케 씀.
click_movie = wd.find_elements_by_css_selector('td.result_text a') #영화 검색창 후 이름 클릭
click_movie[0].click() #개별 페이지 진입
time.sleep(1)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


8.5


In [0]:
# 선택자 실험실- request로 확인 +추후 try except각각 처리해주기 필수 . (무시하셔도 됩니다!)
#Movie_title,Year,Rate, Certificate, runtime, Director, Writers, Stars,Certificate, Language,Released_Date. runtime은 다시.(영화페이지 밑에 분으로 나온놈 있음.)-->abort. coz N/A in some pages.

#조커페이지 입장
url = "https://www.imdb.com/find?q="+str("Joker")+"&ref_=nv_sr_sm" #movie_title에 영화 이름 1개 들어가야함. 
wd.get(url)
  #print(wd.page_source)  # results
click_movie = wd.find_elements_by_css_selector('td.result_text a') #영화 검색창 후 이름 클릭
click_movie[0].click()
import time
time.sleep(1)
  # 여기부터 영화 개별 웹페이지 진입 성공, 자료수집  



# = find_element_by_css_selector("").text 이거뭐임?*
 


#Year(released year)
Year = wd.find_element_by_css_selector("span#titleYear a").text


#Rate(전체 별점) 
Rate =  html.select_one("div.ratingValue span:nth-of-type(1):first-child").text.strip() # nth-of-type(n번째) n은 1부터 시작, first-child하면 부모 모든 요소 중 첫번째 자식만 선택 
print(Rate)

#Certificate(제한 연령),Relased date * US거로 다시하기. 밑에서
  #container = html.select("div.subtext")
#Run time
Run_time = html.select_one("div.subtext time").text.strip()
print(Run_time)

#Genre - 수집 중 날짜까지 리스트 마지막 수집되어서 index[-1]지우는 식으로 코딩함.
genre = html.select("div.subtext a")[0:-1]
Genre = []
for gen in range(len(genre)):
  Genre.append(genre[gen].text.strip())
#print(Genre)

# Director, Writer, Starring div.credit_summary_item줄별로. 
Container = html.select("div.credit_summary_item")

#Director
Director = []
director = Container[0].select("a") 
for i in range(len(director)):
  Director.append(director[i].text.strip())
print(Director)

  # n more credits 지우기
for i in range(len(Director)):
  if "more credits" or "more credit" in Director[i]:
   Ddelete_i = i # Director에서 지워야하는 인덱스
  else: 
    pass
del Director[Ddelete_i] 


#자동화하기(여러 페이지용.)

#Certificate > clicking see more button using Selenium
url = "https://www.imdb.com/title/"+movie_code+"/parentalguide?ref_=tt_stry_pg#certification" #개별영화 페이지 movie_title에 영화 이름 1개 들어가야함. 
wd.get(url)
#선택자 div.txt-block>span.see-more.inline>a

Certificate_Button = wd.find_element_by_css_selector("div.txt-block>span.see-more.inline>a") #클릭됨?확인
Certificate_Button.click()
time.sleep(1)
#컨테이너 li.ipl-inline-list__item a
countries = wd.find_elements_by_css_selector("li.ipl-inline-list__item a")
for i in range(len(countries)):
  if "United States" in countries[i].text:
    cert = countries[i].text
    cert = cert.split(":")
    Certificate = cert[1]
    #print(Certificate)
    #여기서 마지막 글자만 가져오는 식으로 하자.


#Language: 로 시작하는 컨테이너찾기, 적당히 인덱싱해서 Language찾기. 
container = html.select("div.txt-block")
for i in range(len(container)):
  if container[i].text.strip().startswith("Language:"):
    Lang = container[i].text.strip()
Language = Lang[9:]
#print(Language)


  # Language인 행에서 text가져오기시도. div.txt-block > h4.inline 가
  container = html.select("div.txt-block > h4.inline")
  for i in range(len(container)):
    if container[i].text.strip() == "Language:":
      print(container[i].text.strip())--> #"Language" 나옴


  #그냥 전처리 없이 language:English만 가져오는법
  container = html.select("div.txt-block > a") # >a하면 안되고 > a하니까 됨..
  for i in range(len(container)):
    if container[i].text.strip() == "English":
      print(container[i].text.strip())

#Relased_date
container = html.select("div.txt-block")
for i in range(len(container)):
  if container[i].text.strip().startswith("Release Date:"):
    RDate_list = container[i]
print(RDate_list)

#click span.see-more.inline 눌러야함.
Container = wd.find_elements_by_css_selector("div.txt-block")
for i in range(len(Container)):
  if container[i].text.strip().startswith("Release Date:"):
    RDate_list = Container[i]
print(RDate_list)
RDate_Container = RDate_list.find_elements_by_css_selector("span.see-more.inline") 
Relased_date = RDate_Container.find_element_by_css_selector("a")
print(Relased_date)
#print(RDate_list)
#Rdate_Button.click()


#Released Date(circumvent)
#이거 클릭하면 들어감. Released date div.subtext>a:last-child 
#주의.위에서 url로 클릭한 상태이기에 url초기화해서 다시해줘야.
url = "https://www.imdb.com/title/tt7286456/?ref_=ttrel_rel_tt" #개별영화 페이지 movie_title에 영화 이름 1개 들어가야함. 
wd.get(url)
Released_year = wd.find_elements_by_css_selector("div.subtext > a") 
Released_year[-1].click()
time.sleep(1)
#국가,개봉일 목록 --> tr.ipl-zebra-list__item.release-date-item 
Date_list = wd.find_elements_by_css_selector("tr.ipl-zebra-list__item.release-date-item")

#USA로 시작하는 칸 찾기, 최초 개봉일 extract

for i in range(len(Date_list)):
  if Date_list[i].text.startswith("USA") and "premiere" in Date_list[i].text: #여러 개봉일 중"premiere"개봉일 추출
     premiere_released = i  #인덱스 찾기 성공, td.release-date-item__date 여기로 연계.
     
Date_list = wd.find_elements_by_css_selector("td.release-date-item__date")
Released_Date = Date_list[premiere_released].text #Released_Date 수집 성공
     
#wd.close
"""
#다 리스트로 만들기하고 df활용해보기**

Min (2003)


In [0]:
""" (무시하셔도 됩니다!)
# 엑셀파일 만들기
file_name = 'workbook.xls'
wb = xlwt.Workbook()
ws_1 = wb.add_sheet('sheet0',cell_overwrite_ok = True)
ws_1 = wb.get_sheet(0)
ws_1.write(1,1,'영화제목')
"""

In [0]:
#Writers,Stars 개별웹페이지에서 바로 가져오는법(See full cast 말고.) (무시하셔도 됩니다!)
"""
# Writers
Writer = []
writer = Container[1].select("a")
for i in range(len(writer)):
  Writer.append(writer[i].text.strip())

  # n more credits 지우기
for i in range(len(Writer)):
  if "more credits" or "more credit" in Writer[i]:
   Wdelete_i = i # Writer에서 지워야하는 인덱스
  else: 
    pass
del Writer[Wdelete_i] 

  # Top2 in Writers 
if len(Writer) > 2: #3명이상이면
  Writer = Writer[0:2]
print(Writer)

# Stars
Stars = []
stars = Container[2].select("a")
for i in range(len(stars)):
  Stars.append(stars[i].text.strip())

  # See full cast & crew 지우기
for i in range(len(Stars)):
  if "See full cast & crew" in Stars[i]:
   Sdelete_i = i # Stars에서 지워야하는 인덱스
  else: 
    pass
del Stars[Sdelete_i] 
  #Top 2 stars
if len(Stars) > 2: #3명이상이면
  Stars = Stars[0:2] # 2명만 extract
print(Stars)
"""

In [0]:
"""(무시하셔도 됩니다!)
#엑셀파일 다운받기
for i in range(len(new_movie_titles)):
  ws_1.write(i+1,1,new_movie_titles[i])
      
      
wb.save(file_name)
files.download(file_name)
"""

In [0]:
#Dataframe으로 저장하기 (무시하셔도 됩니다!)

#pandas 활용해보기
df2 = pd.DataFrame([{} #컬럼:정보 

ValueError: ignored

In [0]:
no1 = [1, 2, 3]
no2 = [11, 22, 33]
no3 = [111, 222, 333]
Columns = ["movie_titles","Genre","Rate","Run_time","Budget","Gross_USA","Language","Writers","Stars","Certification"]
df2 = pd.DataFrame(columns=Columns)
for i in range(len(movie_titles)):
  df2.loc[i] = movie_titles[i],Genre,Rate,Run_time,Budget,Gross_USA,Language,Writers,Stars,Certification #1번째 영화, 그에 해당하는 데이터

print(df2)
df2.to_csv("movie_titles_pilot.csv")

                                    movie_titles      Genre  ... Stars Certification
0                                 Aquaman (2018)  [Musical]  ...    []            []
1                      Verdi: La Traviata (2018)  [Musical]  ...    []            []
2           Deontay Wilder vs. Tyson Fury (2018)  [Musical]  ...    []            []
3                            Stan & Ollie (2018)  [Musical]  ...    []            []
4   Heading Home: The Tale of Team Israel (2018)  [Musical]  ...    []            []
..                                           ...        ...  ...   ...           ...
66                             Take Point (2018)  [Musical]  ...    []            []
67                             Second Act (2018)  [Musical]  ...    []            []
68              The House That Jack Built (2018)  [Musical]  ...    []            []
69      Spider-Man: Into the Spider-Verse (2018)  [Musical]  ...    []            []
70                            The Charmer (2018)  [Musical]  ... 